In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
currdir = os.getcwd()
regex_name = r'(([A-Z])\w+)'
batchrun = '600s-run1'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

## Checking out the Excel sheet (Obtaining list of columns)

In [4]:
filename = currdir + '/experiments/sim-results/d450_ab_r1.xls'
df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

In [5]:
df_tmp.head()

,t,0x20 - approach2_lane3,0x7 - approach1_lane2,0x25 - approach1_lane1,0x19 - approach3_lane3,0x32 - approach0_lane3,0x2 - approach0_lane2,0x28 - approach1_lane3,0x17 - exit0_lane1,0x1d - approach2_lane1,...,0x29 - approach1_lane3,0x15 - exit1_lane1,0x10 - exit3_lane3,0x12 - exit2_lane3,0x13 - exit2_lane1,0x14 - exit1_lane3,0x2e - approach0_lane1,0x18 - approach3_lane3,0x2c - approach0_lane1,0x2a - approach1_lane3
0,60,18.965854,9.926012,31.023360,38.563200,52.529400,28.805166,6.868286,51.840000,29.597538,...,29.008800,51.840000,45.641739,60.480000,41.040000,53.712000,33.340235,0.000000,0.000000,44.137180
1,120,18.965854,5.423756,24.712235,35.596800,51.317731,22.548369,5.827636,63.360000,11.032774,...,27.327130,51.840000,46.656000,51.840000,47.520000,53.712000,11.808000,14.362667,1.391706,43.104414
2,180,22.007547,4.115871,17.953333,26.534312,51.800949,19.025982,6.221859,63.803077,3.599020,...,27.074880,51.840000,48.028235,45.360000,46.512000,59.680000,4.239222,8.617600,1.163315,41.209714
3,240,22.007547,3.600000,17.019085,27.859346,50.726241,14.987077,6.739344,61.144615,3.599020,...,26.940179,51.840000,47.576842,48.195000,47.424000,59.426043,4.239222,8.617600,1.038378,40.486737
4,300,12.888398,3.114354,16.399164,27.245739,49.947647,9.332521,6.214163,59.560851,2.317880,...,21.650542,54.568421,48.490787,47.421818,46.460377,56.047304,2.732587,9.694800,1.088712,39.530800


In [6]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

In [7]:
# Filter out some of the columns

survey_zones = df_tmp.columns.values[1:]

entrances = [x for x in survey_zones if 'approach' in x]
exits = [y for y in survey_zones if 'exit' in y]

print("{} approach zones, {} exit zones".format(len(entrances), len(exits)))

48 approach zones, 12 exit zones


In [8]:
reg_approach = re.compile('0x(\w+) - approach.*')
reg_exit = re.compile('0x(\w+) - exit.*')

entrances_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_approach.match(k).group(1)])
    for k in entrances
])

exits_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_exit.match(k).group(1)])
    for k in exits
])

## Checking out the Volume sheet

In [9]:
filename = currdir + '/tmp/result_d600_a0.5_b0.5_c0.xls'
df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)

In [10]:
df_tmp

,t,0x20 - approach2_lane3,0x21 - approach2_lane3,0x22 - approach2_lane3,0x23 - approach2_lane3,0x2c - approach0_lane1,0x2d - approach0_lane1,0x2e - approach0_lane1,0x2f - approach0_lane1,0x34 - approach3_lane2,...,0x8 - approach1_lane2,0x9 - approach1_lane2,0xa - approach1_lane2,0x1c - approach2_lane1,0x1d - approach2_lane1,0x1e - approach2_lane1,0x1f - approach2_lane1,0x14 - exit1_lane3,0xb - exit2_lane2,0x6 - exit1_lane2
0,60,5.000000,6.000000,6.000000,7.000000,5.000000,5.000000,5.000000,5.000000,18.000000,...,20.000000,21.000000,22.000000,4.000000,7.000000,9.000,10.000000,5.000000,3.000000,14.000000
1,120,12.000000,11.000000,11.000000,10.000000,4.000000,8.000000,9.000000,10.000000,20.000000,...,25.000000,25.000000,25.000000,3.000000,4.000000,6.000,9.000000,12.000000,15.000000,16.000000
2,180,6.000000,6.000000,6.000000,6.000000,3.000000,3.000000,6.000000,8.000000,16.000000,...,19.000000,19.000000,18.000000,3.000000,3.000000,3.000,3.000000,6.000000,16.000000,16.000000
3,240,6.000000,7.000000,7.000000,8.000000,3.000000,3.000000,3.000000,4.000000,16.000000,...,11.000000,10.000000,12.000000,3.000000,3.000000,3.000,3.000000,6.000000,20.000000,16.000000
4,300,10.000000,9.000000,9.000000,8.000000,3.000000,3.000000,3.000000,3.000000,16.000000,...,20.000000,20.000000,18.000000,3.000000,3.000000,3.000,3.000000,10.000000,15.000000,16.000000
5,360,9.000000,10.000000,11.000000,11.000000,3.000000,3.000000,3.000000,3.000000,16.000000,...,20.000000,21.000000,23.000000,3.000000,3.000000,3.000,3.000000,9.000000,20.000000,16.000000
6,Average Volume,0.133333,0.136111,0.138889,0.138889,0.058333,0.069444,0.080556,0.091667,0.283333,...,0.319444,0.322222,0.327778,0.052778,0.063889,0.075,0.086111,0.133333,0.247222,0.261111
7,Total,48.000000,49.000000,50.000000,50.000000,21.000000,25.000000,29.000000,33.000000,102.000000,...,115.000000,116.000000,118.000000,19.000000,23.000000,27.000,31.000000,48.000000,89.000000,94.000000


In [17]:
vol_rows = [d for d in range(60,361,60)]

df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
df_volume

## ADD THIS LATER WHEN RESULTS ARE OUT

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,5.0,6.0,4.0,6.0,6.0,19.0,22.0,18.0,10.0,10.0,...,44.0,44.0,23.0,38.0,42.0,41.0,23.0,37.0,39.0,39.0
120,4.0,4.0,3.0,6.0,18.0,22.0,20.0,20.0,14.0,10.0,...,46.0,47.0,41.0,39.0,41.0,46.0,39.0,38.0,38.0,44.0
180,3.0,8.0,3.0,8.0,16.0,20.0,20.0,16.0,5.0,10.0,...,28.0,41.0,28.0,39.0,31.0,39.0,25.0,39.0,30.0,37.0
240,3.0,10.0,3.0,7.0,22.0,13.0,20.0,16.0,12.0,10.0,...,28.0,37.0,36.0,28.0,25.0,38.0,35.0,29.0,26.0,36.0
300,3.0,8.0,3.0,7.0,13.0,20.0,20.0,16.0,4.0,11.0,...,31.0,34.0,21.0,43.0,33.0,31.0,22.0,42.0,34.0,30.0
360,3.0,10.0,3.0,7.0,21.0,20.0,20.0,16.0,10.0,11.0,...,27.0,33.0,36.0,41.0,28.0,31.0,36.0,40.0,28.0,31.0


## Preprocessing the results

In [13]:
weight_keywords = {
    'a': (0.8, 0.1, 0.1),
    'b': (0.1, 0.8, 0.1),
    'c': (0.1, 0.1, 0.8),
    'ab': (0.45, 0.45, 0.1),
    'bc': (0.1, 0.45, 0.45),
    'ac': (0.45, 0.1, 0.45),
    'abc': (0.33, 0.33, 0.33),
    'old': (0, 0, 0),
}

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_model_type = {
    (0.8, 0.1, 0.1): 'Delay priority',
    (0.1, 0.8, 0.1): 'Throughput priority',
    (0.1, 0.1, 0.8): 'Flow priority',
    (0.45, 0.45, 0.1): 'Delay-Throughput priority',
    (0.1, 0.45, 0.45): 'Throughput-Flow priority',
    (0.45, 0.1, 0.45): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [15]:
total_thru_dfs = []
total_delay_dfs = []
ave_delay_dfs = []

for demand in demands:
    for kw, weights in weight_keywords.iteritems():

        filename = currdir + '/experiments/sim-results/d{}_{}_r1.xls'.format(demand, kw)
        df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

        # Get the cumulative delays

        df_tmp1 = df_tmp[df_tmp.t.isin(delay_rows)]
        df_tmp1['t'].replace(delay_rename, inplace=True)
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_cumdelay = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_cumdelay.to_pickle(currdir + '/experiments/cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        
        # Get the cumulative throughputs

        df_tmp3 = df_tmp[df_tmp.t.isin(thru_rows)]
        if kw == 'old':
            for col in ['0x11 - exit3_lane1', '0x15 - exit1_lane1', '0x17 - exit0_lane1', '0x13 - exit2_lane1']:
                df_tmp3[col] = 0
        df_tmp3['t'].replace(thru_rename, inplace=True)
        df_tmp3 = df_tmp3.sort_values(by='t').set_index('t')[exits].stack().reset_index()
        df_tmp3['level_1'] = df_tmp3['level_1'].map(exits_rename)
        df_tmp3 = df_tmp3.groupby(['t', 'level_1']).sum().reset_index()

        df_cumthru = df_tmp3.pivot(index='t', columns='level_1', values=0)
        df_cumthru.to_pickle(currdir + '/experiments/cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))

        # Get aggregate values

        df_total_thru_tmp = df_cumthru.loc[600, :].reset_index().set_index('level_1').T
        df_total_thru_tmp['demand'] = [demand]
        df_total_thru_tmp['new_model'] = kw != 'old'
        df_total_thru_tmp['alpha'] = weights[0]
        df_total_thru_tmp['beta'] = weights[1]
        df_total_thru_tmp['gamma'] = weights[2]
        df_total_thru_tmp['model_type'] = _model_type[weights]
        total_thru_dfs.append(df_total_thru_tmp)

        df_total_delay_tmp = df_cumdelay.loc[600, :].reset_index().set_index('level_1').T
        df_total_delay_tmp['demand'] = [demand]
        df_total_delay_tmp['new_model'] = kw != 'old'
        df_total_delay_tmp['alpha'] = weights[0]
        df_total_delay_tmp['beta'] = weights[1]
        df_total_delay_tmp['gamma'] = weights[2]
        df_total_delay_tmp['model_type'] = _model_type[weights]
        total_delay_dfs.append(df_total_delay_tmp)

        df_tmp2 = df_tmp[df_tmp.t == 'Average Delay'][entrances].rename(columns=entrances_rename).stack().reset_index()
        df_ave_delay_tmp = df_tmp2.groupby('level_1').mean().reset_index().pivot(index='level_0', columns='level_1', values=0)
        df_ave_delay_tmp['demand'] = [demand]
        df_ave_delay_tmp['new_model'] = kw != 'old'
        df_ave_delay_tmp['alpha'] = weights[0]
        df_ave_delay_tmp['beta'] = weights[1]
        df_ave_delay_tmp['gamma'] = weights[2]
        df_ave_delay_tmp['model_type'] = _model_type[weights]
        ave_delay_dfs.append(df_ave_delay_tmp)

/home/franciszacdlc/.virtualenvs/localsim2.7/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
df_total_thru = pd.concat(total_thru_dfs).reset_index().iloc[:, 1:]
df_total_thru.to_pickle(currdir + '/experiments/total_throughput.pkl')

df_total_delay = pd.concat(total_delay_dfs).reset_index().iloc[:, 1:]
df_total_delay.to_pickle(currdir + '/experiments/total_delay.pkl')

df_ave_delay = pd.concat(ave_delay_dfs).reset_index().iloc[:, 1:]
df_ave_delay.to_pickle(currdir + '/experiments/ave_delay.pkl')